In [2]:
%reload_ext autoreload
%autoreload 2

import os
import pprint
import shutil
import pymongo

from climatools.cliradlw import (setup, pipeline)
import climatools.cliradlw.runrecord as runrecord_clirad
import climatools.cliradlw.utils as utils_clirad

from climatools.lblnew import setup_overlap
from climatools.lblnew import setup_bestfit
import climatools.lblnew.pipeline as pipeline_lblnew
import climatools.lblnew.bestfit_params as bestfit
import climatools.lblnew.runrecord_bestfit as runrecord_bestfit
import climatools.lblnew.runrecord_overlap as runrecord_overlap

In [93]:
client = pymongo.MongoClient('localhost', 27017)

In [94]:
client.cliradnew.lw.count()

234

In [75]:
ATMPROS = [('mls', 294), ('saw', 257), ('trp', 300)]

In [258]:
# EDIT HERE TO START

def params_clirad_nonoverlap():
    band = 11
    
    from climatools.atm.absorbers import nongreys_byband
    atmpros = [('mls', 294), ('saw', 257), ('trp', 300)]
    commitnumber = '1013f91'
    molconc = nongreys_byband()[band]
    params = [{'commitnumber': commitnumber,
               'band': [band],
               'molecule': {mol: conc},
               'atmpro': atmpro,
               'tsfc': tsfc} 
              for mol, conc in molconc.items()
              for atmpro, tsfc in atmpros]
    
#    if len(molconc) > 1:
#        params.extend(
#            [{'commitnumber': commitnumber, 
#             'band': [band], 
#              'molecule': molconc, 
#              'atmpro': atmpro, 
#              'tsfc': tsfc} for atmpro, tsfc in atmpros])
    return params

In [259]:
def param_clirad2lblnewbestfit(param=None, commitnumber=None):
    '''
    Concert clirad param to lblnew-bestfit partial param
    
    Parameters
    ----------
    param: dict
        clirad's param dictionary
    commitnumber: str
        Git commit number of the latest lblnew-bestfit version
    '''
    molecule, conc = list(param['molecule'].items())[0]
    band = param['band'][0]
    band = utils_clirad.mapband_new2old()[band]
    atmpro = param['atmpro']
    tsfc = param['tsfc']
    return {'commitnumber': commitnumber,
            'molecule': molecule, 'conc': conc,
            'band': band, 
            'atmpro': atmpro, 'tsfc': tsfc}

In [260]:
def to_lblnewparam(param=None, commitnumber=None):
    '''
    Convert clirad's input parameter dictionary 
    into something like lblnew's.
    '''
    # Unpack clirad-lw's param
    molecule, conc = list(param['molecule'].items())[0]
    conc = None if conc == 'atmpro' else conc
    band = param['band'][0]
    band = utils_clirad.mapband_new2old()[band]
    atmpro = param['atmpro']
    tsfc = param['tsfc']
    return {'band': band,
            'molecule': molecule, 'conc': conc,
            'dv': .001, 'nv': 1000,
            'atmpro': atmpro, 'tsfc': tsfc}

def make_query(param=None)   :
    return {f'param.{n}': v for n, v in param.items()}

def get_parampairs():
    pairs = []
    for p in params_clirad_nonoverlap():
        
        q_clirad = make_query(param=p)
        r_clirad = client.cliradnew.lw.count(q_clirad)
        if not r_clirad: 
            print('Following clirad case not available:')
            pprint.pprint(p)
            print('Skipping it.\n')
            continue
        
        p_lblnew = to_lblnewparam(param=p)
        q_lblnew = make_query(param=p_lblnew)
        r_lblnew = client.lblnew.bestfit_lw.find_one(q_lblnew, projection=['param'])
        if not r_lblnew:
            print('Following lblnew-bestfit case not available:')
            pprint.pprint(p_lblnew)
            print('Skipping it.\n')
            continue
            
        pairs.append((p, r_lblnew.get('param')))
    return pairs

In [262]:
get_parampairs()

[({'atmpro': 'mls',
   'band': [11],
   'commitnumber': '1013f91',
   'molecule': {'co2': 0.0004},
   'tsfc': 294},
  {'atmpro': 'mls',
   'band': '9',
   'commitnumber': 'a22ab94',
   'conc': 0.0004,
   'dv': 0.001,
   'klin': 6.5e-24,
   'molecule': 'co2',
   'ng_adju': [0, 0],
   'ng_refs': [3, 3],
   'nv': 1000,
   'option_compute_btable': 0,
   'option_compute_ktable': 0,
   'option_wgt_flux': 1,
   'option_wgt_k': 1,
   'ref_pts': [[1, 250], [50, 250]],
   'tsfc': 294,
   'vmax': 3000,
   'vmin': 1900,
   'w_diffuse': [[1.66, 1.66, 1.75], [1.75, 1.6, 1.85]],
   'wgt': [[0.7, 0.8, 0.7], [0.8, 0.7, 0.8]]}),
 ({'atmpro': 'saw',
   'band': [11],
   'commitnumber': '1013f91',
   'molecule': {'co2': 0.0004},
   'tsfc': 257},
  {'atmpro': 'saw',
   'band': '9',
   'commitnumber': 'a22ab94',
   'conc': 0.0004,
   'dv': 0.001,
   'klin': 6.5e-24,
   'molecule': 'co2',
   'ng_adju': [0, 0],
   'ng_refs': [3, 3],
   'nv': 1000,
   'option_compute_btable': 0,
   'option_compute_ktable': 0,
 

In [255]:
def script_clirad_lblnew():
    '''
    Script to run clirad and then plot results against the 
    corresponding lblnew case.  This needs both input parameter
    dictionaries from clirad and lblnew.
    '''
    ps = get_parampairs()
    gprocs = pipeline.pipeline_ipynb2git(parampairs=ps, setup=setup)

In [263]:
script_clirad_lblnew()

{'atmpro': 'mls',
 'band': [11],
 'commitnumber': '1013f91',
 'molecule': {'co2': 0.0004},
 'tsfc': 294}
{'atmpro': 'saw',
 'band': [11],
 'commitnumber': '1013f91',
 'molecule': {'co2': 0.0004},
 'tsfc': 257}
{'atmpro': 'trp',
 'band': [11],
 'commitnumber': '1013f91',
 'molecule': {'co2': 0.0004},
 'tsfc': 300}
{'atmpro': 'mls',
 'band': [11],
 'commitnumber': '1013f91',
 'molecule': {'h2o': 'atmpro'},
 'tsfc': 294}
{'atmpro': 'saw',
 'band': [11],
 'commitnumber': '1013f91',
 'molecule': {'h2o': 'atmpro'},
 'tsfc': 257}
{'atmpro': 'trp',
 'band': [11],
 'commitnumber': '1013f91',
 'molecule': {'h2o': 'atmpro'},
 'tsfc': 300}
{'atmpro': 'mls',
 'band': [11],
 'commitnumber': '1013f91',
 'molecule': {'co2': 0.0004},
 'tsfc': 294}
{'atmpro': 'saw',
 'band': [11],
 'commitnumber': '1013f91',
 'molecule': {'co2': 0.0004},
 'tsfc': 257}
{'atmpro': 'trp',
 'band': [11],
 'commitnumber': '1013f91',
 'molecule': {'co2': 0.0004},
 'tsfc': 300}
{'atmpro': 'mls',
 'band': [11],
 'commitnumber':

In [264]:
for param, _ in get_parampairs():
    print(pipeline.nbviewer_url(param=param, setup=setup))
    print()

http://nbviewer.jupyter.org/github/qap/analysis_-_new_kdist_param/blob/master/clirad/co2_0.0004/band_11/atmpro_mls/cliradlw_1013f91/results_cliradlw.ipynb

http://nbviewer.jupyter.org/github/qap/analysis_-_new_kdist_param/blob/master/clirad/co2_0.0004/band_11/atmpro_saw/cliradlw_1013f91/results_cliradlw.ipynb

http://nbviewer.jupyter.org/github/qap/analysis_-_new_kdist_param/blob/master/clirad/co2_0.0004/band_11/atmpro_trp/cliradlw_1013f91/results_cliradlw.ipynb

http://nbviewer.jupyter.org/github/qap/analysis_-_new_kdist_param/blob/master/clirad/h2o_mls/band_11/atmpro_mls/cliradlw_1013f91/results_cliradlw.ipynb

http://nbviewer.jupyter.org/github/qap/analysis_-_new_kdist_param/blob/master/clirad/h2o_saw/band_11/atmpro_saw/cliradlw_1013f91/results_cliradlw.ipynb

http://nbviewer.jupyter.org/github/qap/analysis_-_new_kdist_param/blob/master/clirad/h2o_trp/band_11/atmpro_trp/cliradlw_1013f91/results_cliradlw.ipynb

